In [1]:
import numpy as np
import random
import pandas as pd
from pandas import DataFrame
from sklearn import svm
from prettytable import PrettyTable

def rand_n_int(l, r, n):
    s=[]
    while(len(s)<n):
        x=random.randint(l,r)
        if x not in s:
            s.append(x)
    return s

label01 = np.load("labels_01.npy")
label01 = label01.reshape(-1,1)

#label_table = pd.read_table("E-TABM-185.sdrf.txt", sep='\t')

#labels = DataFrame(label_table, columns=['Is Disease'])

#labels_normal = labels[labels['Is Disease'] == 0]
#labels_cancer = labels.drop(labels_normal.index)

#label01 = np.zeros(label_table.iloc[:,0].size)
#for i in range(label_table.iloc[:,0].size):
#    if i in labels_cancer.index:
#        label01[i] = 1
data = np.load("data_reduce.npy")
#data = pd.read_table("microarray.original.txt", sep='\t')
#data = data.drop(['ProbeSet ID'], axis = 1).values
#data = data.T

train_index = np.array(rand_n_int(0, data.shape[0]-1, 4000))
test_index = []
for i in range(data.shape[0]):
    if i not in train_index:
        test_index.append(i)
test_index = np.array(test_index)

data_train = data[train_index,:]
label_train = label01[train_index]
data_test = data[test_index,:]
label_test = label01[test_index]

In [2]:
clf = svm.SVC()
clf.fit(data_train, label_train)

D:\Python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [3]:
correct = 0
correct, correct1, correct0, false1, false0 = 0,0,0,0,0
for i in range(test_index.shape[0]):
    res = clf.predict(data_test[i,:].reshape(1, -1))
    if res == label_test[i]:
        correct+=1
        if res == 1:
            correct1+=1
        else:
            correct0+=1
    else:
        if res == 1:
            false1+=1
        else:
            false0+=1
print("Accuracy: %.4f" % (correct/test_index.shape[0]))
tab = PrettyTable()
tab.field_names = ["", "Cancer","Normal","Precision"]

tab.add_row(['Predict Cancer',correct1,false1,"%.4f"%(correct1/(correct1+false1))])
tab.add_row(['Predict Normal',false0,correct0,"%.4f"%(correct0/(correct0+false0))])
tab.add_row(['Recall',"%.4f"%(correct1/(correct1+false0)), "%.4f"%(correct0/(false1+correct0)),""])
print(tab)

Accuracy: 0.9346
+----------------+--------+--------+-----------+
|                | Cancer | Normal | Precision |
+----------------+--------+--------+-----------+
| Predict Cancer |  1238  |   90   |   0.9322  |
| Predict Normal |   34   |  534   |   0.9401  |
|     Recall     | 0.9733 | 0.8558 |           |
+----------------+--------+--------+-----------+
